# HW 4

## Imports

In [1]:
import nbtools
nbtools.setup_nb()

In [2]:
import itertools
import sympy
from sympy.diffgeom import Manifold, Patch
from pystein import coords, metric, curvature
from pystein.utilities import tensor_pow as tpow, full_simplify

## Problem 1

### Problem 1.1

In [16]:
def spherical_laplacian_1d(func, param):
    inner = param ** 2 * sympy.Derivative(func, param)
    return 1 / param ** 2 * sympy.Derivative(inner, param)

In [22]:
r = sympy.symbols('r')

# psi = sympy.Function('psi')(r)
psi_exp = r ** 2 / (1 + r ** 3)
psi_exp

r**2/(r**3 + 1)

In [20]:
lhs = full_simplify(spherical_laplacian_1d(psi_exp, r).doit())

$$\LARGE g(r)$$

In [36]:
g = full_simplify(lhs / full_simplify(psi_exp ** 4))
g

6*(-2*r**6 - r**3 + 1)/r**8

### Problem 1.2

$$\LARGE\nabla^2 \psi - \psi^4 g(r) = 0 \equiv \frac{F}{r^2} \equiv f$$

$$\LARGE \frac{1}{r^2}\partial_r \left( r^2 \partial_r \psi \right) - 6\left(\frac{-2r^6 -r^3 + 1}{r^8}\right) \psi^4  = 0 \equiv \frac{F}{r^2} \equiv f$$

$$\LARGE F = \partial_r \left( r^2 \partial_r \psi \right) - 6\left(\frac{-2r^6 -r^3 + 1}{r^6}\right) \psi^4  = 0$$

$$\LARGE F = \partial_r \left( r^2 \partial_r \psi \right) - r^2 g(r) \psi^4  = 0$$

$$\LARGE J_{ij}(\psi) \equiv \frac{\partial}{\partial \psi_j} G_i (\psi)$$

$$\LARGE \Phi \equiv \frac{\partial F}{\partial \psi} = ? - 4 r^2 g(r) \psi^3$$

$$\LARGE A \equiv \delta \psi$$

In [54]:
psi = sympy.Function('psi')(r)
d_psi = sympy.Function('\\delta \\psi')(r)
psi_pert = psi + d_psi
g = sympy.symbols('g')

In [59]:
def F(func, param):
    return r ** 2 * spherical_laplacian_1d(func, param) - r ** 2 * g * func ** 4

In [60]:
rhs = F(psi_pert, r) - F(psi, r)
rhs = full_simplify(rhs.doit())
rhs

r*(-g*r*\delta \psi(r)**4 - 4*g*r*\delta \psi(r)**3*psi(r) - 6*g*r*\delta \psi(r)**2*psi(r)**2 - 4*g*r*\delta \psi(r)*psi(r)**3 + r*Derivative(\delta \psi(r), (r, 2)) + 2*Derivative(\delta \psi(r), r))

In [61]:
first_order_subs = [
    (d_psi ** 2, 0),
    (d_psi ** 3, 0),
    (d_psi ** 4, 0),
]

In [62]:
full_simplify(sympy.expand(rhs).subs(first_order_subs))

r*(-4*g*r*\delta \psi(r)*psi(r)**3 + r*Derivative(\delta \psi(r), (r, 2)) + 2*Derivative(\delta \psi(r), r))

$$\LARGE A \Phi = - 4 g r^2 \delta \psi \psi^3 + r^2 \partial^2_r \delta \psi + 2 r \partial_r \delta \psi$$

$$\LARGE A \Phi = - 4 g r^2 \delta \psi \psi^3 + \partial_r \left(r^2 \partial_r \delta \psi\right)$$

Notes about translating to code:

- $A \Phi$ goes into matvec
- $\psi$ is `self.up`
- $\delta \psi$ is `Phi_`

### Problem 1.2

In [68]:
M = Manifold('M', dim=3)
P = Patch('origin', M)
rho, z, phi = sympy.symbols('rho z phi', nonnegative=True)

psi = sympy.Function('psi')(rho, z, phi)
q = sympy.Function('q')(rho, z)
cs = coords.CoordSystem('cylindrical', P, [rho, z, phi])
drho, dz, dphi = cs.base_oneforms()


ds2 =  (sympy.exp(q) * (tpow(drho, 2) + tpow(dz, 2)) 
                  + rho ** 2 * tpow(dphi, 2))
gamma = metric.Metric(twoform=ds2)

gamma

rho**2*TensorProduct(dphi, dphi) + exp(q(rho, z))*(TensorProduct(drho, drho) + TensorProduct(dz, dz))

In [69]:
ricci = curvature.ricci_scalar(gamma).doit()
ricci = full_simplify(ricci)
ricci

-(Derivative(q(rho, z), (rho, 2)) + Derivative(q(rho, z), (z, 2)))*exp(-q(rho, z))

In [70]:
christoffels = [((i, i, k), curvature.christoffel_symbol_component(k, i, i, gamma).doit()) 
                for i, k in itertools.product(range(3), range(3))]

In [71]:
curvature.display_components(christoffels)

LHS

In [72]:
one_form = [sympy.Derivative(psi, cs.base_symbols()[k]) for k in range(3)]

In [73]:
def cov_deriv_one_form(i, j, form, g, scale_by_inverse: bool = False):
    coord_syms = g.coord_system.base_symbols()
    v = sympy.Derivative(form[j], coord_syms[i])
    
    for k in range(len(coord_syms)):
        term = curvature.christoffel_symbol_component(k, i, j, g) * form[k]
        if scale_by_inverse:
            term *= g.matrix.inverse()[i, j]
        v -= term

    return full_simplify(v.doit())

In [74]:
def Dbar2(i, j, conformal_func, g):
    one_form = [sympy.Derivative(psi, g.coord_system.base_symbols()[k]) for k in range(3)]
    raw = cov_deriv_one_form(i, j, one_form, g)
    return full_simplify(raw.doit())

In [75]:
DiDjPsi = sympy.Matrix([[Dbar2(0, j, psi, gamma) for j in range(3)],
                            [Dbar2(1, j, psi, gamma) for j in range(3)],
                            [Dbar2(2, j, psi, gamma) for j in range(3)]])

In [76]:
Dbar2Psi = sum([gamma.matrix.inverse()[i, j] * DiDjPsi[i, j] 
                for i, j in itertools.product(range(3), range(3))])

In [77]:
LHS = full_simplify(Dbar2Psi)
RHS = psi / 8 * ricci

In [78]:
notation_subs = [
    (psi, sympy.symbols('psi'))
]

In [79]:
LHS.subs(notation_subs)

(rho**2*(Derivative(psi, (rho, 2)) + Derivative(psi, (z, 2))) + rho*Derivative(psi, rho) + exp(q(rho, z))*Derivative(psi, (phi, 2)))*exp(-q(rho, z))/rho**2

In [80]:
RHS.subs(notation_subs)

-psi*(Derivative(q(rho, z), (rho, 2)) + Derivative(q(rho, z), (z, 2)))*exp(-q(rho, z))/8

In [50]:
def flat_laplacian(func):
    return ((1 / rho) * sympy.Derivative((rho * sympy.Derivative(func, rho)), rho)
             + (1 / rho) ** 2 * sympy.Derivative(sympy.Derivative(func, phi), phi)
             + sympy.Derivative(sympy.Derivative(func, z), z))

In [67]:
flat_laplacian(psi).doit().subs(notation_subs)

Derivative(psi, (z, 2)) + (rho*Derivative(psi, (rho, 2)) + Derivative(psi, rho))/rho + Derivative(psi, (phi, 2))/rho**2

In [81]:
gamma.matrix.inverse()

Matrix([
[exp(-q(rho, z)),               0,         0],
[              0, exp(-q(rho, z)),         0],
[              0,               0, rho**(-2)]])

In [ ]:
cov_deriv_one_form()